# 8 Queens

### Introduction

<figure>
<img src="resources/eight_queens_moves.png", width=300 align="right">
    <figcaption></figcaption>
</figure>

The [Eight Queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a famous puzzle that has been studied extensively in- and outside of computer science. It was first published in the chess magazine _Schach_ in 1848. 

The problem can be formulated as follows: 

_"Place 8 queens on a regular (8x8) chess board such that no queen attacks any other queen."_

A queen in the game of chess can move horizontally, vertically, and diagonally. The puzzle can be solved by hand (and even [Carl Friedrich Gauss](https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss) studied it back in 1850).

The EightQueensState class below, as well as the methods defined, should prove a helpful start for a Genetic Algorithms approach. However, you are welcome to change as little or as much of the code as is useful.

In [1]:
import numpy as np

class EightQueensState:
    """This class represents a board in the eight queens puzzle"""
    def __init__(self, state=None, n=8):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        if state is None:
            self.n = n
            state = np.random.randint(0, n, n)
        else:
            self.n = len(state)
        self.state = state

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))

    def cost(self):
        """Calculates the number of pairs attacking"""
        count = 0
        for i in range(len(self.state) - 1):
            # for each queen, look in columns to the right
            # add one to the count if there is another queen in the same row
            count += (self.state[i] == np.array(self.state[i + 1:])).sum()

            # add one to the count for each queen on the upper or lower diagonal
            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += (np.array(self.state[i + 1:]) == upper_diagonal).sum()
            count += (np.array(self.state[i + 1:]) == lower_diagonal).sum()
        return count
    
    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)

    def is_goal(self):
        return self.cost() == 0

    def __str__(self):
        if self.is_goal():
            return f"Goal state! {self.state}"
        else:
            return f"{self.state} cost {self.cost()}"

In [218]:
import random
from datetime import datetime
import statistics as stats
import copy

In [331]:
def get_random_pop(n_ind, n_queens):
    # get a random population of n individual states for games of n queens
    population = []
    for i in range(n_ind):
        population.append(EightQueensState(n = n_queens))
    return population

In [332]:
# Using pseudo code from Russel and Norvig, 2021, p. 133
def reproduce(parent1, parent2):
    """
    :params p1 and p2 are the EightQueensState objects of parents 1 and 2
    """
    n = len(parent1.state)
    c = np.random.choice(n) # random crossover point
    child1 = np.append(parent1.state[:c], parent2.state[c:])
    child2 = np.append(parent2.state[:c], parent1.state[c:])
    
    mutation_rate = 1/n
    for i in range(len(child1)):
        if random.random() < mutation_rate:
            child1[i] = random.randint(0,7)
    
    for i in range(len(child2)):
        if random.random() < mutation_rate:
            child2[i] = random.randint(0,7)
    
    return EightQueensState(child1), EightQueensState(child2)

In [333]:
# Using pseudo code from Russel and Norvig, 2021, p. 133
def GA_solver1(population):
    """
    :param population contains a population of individual game states, represented as a list of numpy arrays
    """
    pop = copy.deepcopy(population)
    fittest_score = 0
    generation = 0
    while fittest_score != 28: # keep generating new populations until the game is solved
        parents = []
        children = []
        fitness = [28 - i.cost() for i in pop]
        fitness_sorted = sorted(fitness)
        
        for i in range(len(fitness)):
            if fitness[i] > fittest_score:
                fittest_score = fitness[i]
                fittest_state = pop[i]
        
        total_fitness = sum(fitness)
        fitness_prop = [i/total_fitness for i in fitness]
        
        for i in range(int(len(pop)/2)):
            parents.append(np.random.choice(pop, size=2, replace=False, p=fitness_prop))
            
        for pair in parents:
            child1, child2 = reproduce(pair[0], pair[1])
            children.extend([child1, child2])
        
        pop = children
        generation += 1
    return fittest_state, generation # returns the 8 Queens class object of the solution and the number of generations it took to get there

In [249]:
for i in range(10):
    p = get_random_pop(10,8)
    start_time = datetime.now()
    solution = GA_solver1(p)
    print(solution.state)
    print(datetime.now() - start_time)

[3 6 4 2 0 5 7 1]
0:00:24.035334
[3 6 4 1 5 0 2 7]
0:00:05.736620
[3 1 7 4 6 0 2 5]
0:00:19.238364
[3 7 4 2 0 6 1 5]
0:01:42.202491
[6 3 1 4 7 0 2 5]
0:00:05.586370
[5 7 1 3 0 6 4 2]
0:00:08.758874
[7 1 4 2 0 6 3 5]
0:00:24.056485
[1 6 4 7 0 3 5 2]
0:00:18.916673
[2 5 3 1 7 4 6 0]
0:00:15.455867
[5 2 0 7 4 1 3 6]
0:00:08.832106


# Implementation 2
N-Queens using culling and elitism.

In [334]:
# for task 2
def reproduce2(parent1, parent2):
    """
    :params p1 and p2 are the EightQueensState objects of parents 1 and 2
    """
    n = len(parent1.state)
    c = np.random.choice(n) # random crossover point
    child1 = np.append(parent1.state[:c], parent2.state[c:])
    child2 = np.append(parent2.state[:c], parent1.state[c:])
    
    mutation_rate = 1/n
    for i in range(len(child1)):
        if random.random() < mutation_rate:
            child1[i] = random.randint(0,n-1)
    
    for i in range(len(child2)):
        if random.random() < mutation_rate:
            child2[i] = random.randint(0,n-1)
    
    return EightQueensState(child1), EightQueensState(child2)

In [335]:
# for task 2
def GA_solver2(population):
    """
    :param population contains a population of individual game states, represented as a list of numpy arrays
    """
    pop = copy.deepcopy(population) # making deepcopy to allow algorithm repition on the same population
    fittest_score = 0
    generation = 0
    n_queens = len(population[0].state)
    if 1 < n_queens < 4:
        print("No solutions for games with 2 or 3 Queens")
        return
    max_score = (n_queens * (n_queens-1))/2
    while fittest_score != max_score:
        parents = []
        children = []
        fitness = [max_score - i.cost() for i in pop]
        fitness_sorted = sorted(fitness)
        
        for i in range(len(fitness)):
            if fitness[i] > fittest_score:
                fittest_score = fitness[i]
                fittest_state = pop[i]
        
        #elitism
        for i in fitness_sorted[int(len(pop)*0.8):]:
            children.append(pop[fitness.index(i)])
            
        #culling the least fit before breeding
        for i in fitness_sorted[:int(len(pop)/5)]: # cull the lowest 20% of states
            pop.pop(fitness.index(i))
            fitness.pop(fitness.index(i))
        
        total_fitness = sum(fitness)
        fitness_prop = [i/total_fitness for i in fitness]
        
        for i in range(int(len(pop)/2)):
            parents.append(np.random.choice(pop, size=2, replace=False, p=fitness_prop))
            
        for pair in parents:
            child1, child2 = reproduce2(pair[0], pair[1])
            children.extend([child1, child2])
        
        pop = children
        generation += 1
    return fittest_state, generation

In [326]:
p = get_random_pop(4, 2)
solution = GA_solver2(p)
print(solution[0].state)

No solutions for games with 2 or 3 Queens


TypeError: 'NoneType' object is not subscriptable

# Test 1: Population sizes

In [298]:
# testing different population sizes for my first implementation
for i in range(2, 50, 5):
    p = get_random_pop(i, 8)
    start_time = datetime.now()
    solution = GA_solver1(p)
    print(f"solution {i}: {solution[0].state}")
    print(datetime.now() - start_time)
    print()

solution 2: [4 2 7 3 6 0 5 1]
0:00:24.381492

solution 7: [5 2 6 3 0 7 1 4]
0:00:07.707946

solution 12: [2 0 6 4 7 1 3 5]
0:00:20.101817

solution 17: [4 7 3 0 2 5 1 6]
0:00:00.055844

solution 22: [4 2 7 3 6 0 5 1]
0:00:35.905507

solution 27: [3 6 4 2 0 5 7 1]
0:00:45.098183

solution 32: [1 4 6 0 2 7 5 3]
0:00:26.975383

solution 37: [5 3 6 0 7 1 4 2]
0:00:14.159813

solution 42: [5 7 1 3 0 6 4 2]
0:00:14.249594

solution 47: [2 7 3 6 0 5 1 4]
0:00:15.528441



In [299]:
# testing different population sizes for my second implementation
for i in range(2, 50, 5):
    p = get_random_pop(i, 8)
    start_time = datetime.now()
    solution = GA_solver2(p)
    print(f"solution {i}: {solution[0].state}")
    print(datetime.now() - start_time)
    print()

solution 2: [6 2 7 1 4 0 5 3]
0:01:41.691692

solution 7: [4 6 1 5 2 0 7 3]
0:00:02.547223

solution 12: [5 1 6 0 2 4 7 3]
0:00:00.517617

solution 17: [3 6 2 7 1 4 0 5]
0:01:32.059359

solution 22: [1 3 5 7 2 0 6 4]
0:00:01.048230

solution 27: [6 4 2 0 5 7 1 3]
0:00:00.271244

solution 32: [2 4 7 3 0 6 1 5]
0:01:58.102976

solution 37: [6 2 0 5 7 4 1 3]
0:00:00.566485

solution 42: [3 7 0 2 5 1 6 4]
0:00:00.290178

solution 47: [4 1 5 0 6 3 7 2]
0:00:01.714448



# Test 2: Implementation 1 vs 2

In [300]:
gens1=[]
gens2=[]
for i in range(10):
    p = get_random_pop(10,8)
    solution1 = GA_solver1(p)
    print(f"GA 1 solution: {solution1[0].state}")
    gens1.append(solution1[1])
    
    solution2 = GA_solver2(p)
    print(f"GA 2 solution: {solution2[0].state}\n")
    gens2.append(solution2[1])
print(f"1st algorithm: Avg generations: {stats.mean(gens1)}, gens {gens1}\n")
print(f"2nd algorithm: Avg generations: {stats.mean(gens2)}, gens {gens2}")

GA 1 solution: [3 1 7 5 0 2 4 6]
GA 2 solution: [6 3 1 4 7 0 2 5]

GA 1 solution: [5 2 6 1 7 4 0 3]
GA 2 solution: [4 7 3 0 2 5 1 6]

GA 1 solution: [4 0 3 5 7 1 6 2]
GA 2 solution: [2 0 6 4 7 1 3 5]

GA 1 solution: [3 5 0 4 1 7 2 6]
GA 2 solution: [4 0 3 5 7 1 6 2]

GA 1 solution: [5 2 6 1 7 4 0 3]
GA 2 solution: [2 4 1 7 0 6 3 5]

GA 1 solution: [2 4 1 7 0 6 3 5]
GA 2 solution: [5 0 4 1 7 2 6 3]

GA 1 solution: [3 1 7 5 0 2 4 6]
GA 2 solution: [2 6 1 7 5 3 0 4]

GA 1 solution: [2 5 3 1 7 4 6 0]
GA 2 solution: [6 3 1 4 7 0 2 5]

GA 1 solution: [5 2 4 7 0 3 1 6]
GA 2 solution: [5 3 6 0 7 1 4 2]

GA 1 solution: [2 6 1 7 5 3 0 4]
GA 2 solution: [5 0 4 1 7 2 6 3]

1st algorithm: Avg generations: 12110.4, gens [799, 7298, 970, 5363, 22975, 2009, 50295, 8041, 21150, 2204]

2nd algorithm: Avg generations: 13055.5, gens [663, 6858, 31, 16162, 62, 152, 1697, 85138, 2646, 17146]


# Changing board size (n-queens)

In [309]:
results = {}
for i in range(4, 10):
    p = get_random_pop(10,i)
    solution = GA_solver2(p)
    print(solution[0].state)
    results[i] = solution[1] # number of generations
print(results)

[1 3 0 2]
[0 3 1 4 2]
[1 3 5 0 2 4]
[0 4 1 5 2 6 3]
[2 4 1 7 5 3 6 0]
[6 4 0 5 8 2 7 3 1]
{4: 12, 5: 3, 6: 23, 7: 25, 8: 134, 9: 94}


In [329]:
results = {}
for i in range(10, 15):
    p = get_random_pop(10,i)
    solution = GA_solver2(p)
    print(solution[0].state)
    print(solution[1])
    results[i] = solution[1] # number of generations
print(results)

KeyboardInterrupt: 